In [ ]:
import warnings
warnings.filterwarnings('ignore')


from pathlib import Path
import requests

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

import matplotlib.pyplot as plt
import seaborn as sns
from talib import RSI, BBANDS, MACD, NATR, ATR

import pandas as pd
import requests
from bs4 import BeautifulSoup

from datetime import datetime
from time import sleep

pd.set_option('display.expand_frame_repr', False)
DATA_STORE = Path('sp.h5')

In [ ]:
# S&P 500 Bestandteile und Änderungen am Index von Wikipedia beziehen
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

table_constituents = soup.find("table", {"id": "constituents"})
table_changes = soup.find("table", {"id": "changes"})

constituents_df = pd.read_html(str(table_constituents))[0]
changes_df = pd.read_html(str(table_changes))[0].dropna()
print(constituents_df.info())
print(changes_df.info())

print(changes_df)
print(constituents_df)

In [ ]:
# S&P500 Monatliche Bestandteile
monthly_constituents = pd.DataFrame(columns=['Date', 'Constituents'])
current_constituents = set(constituents_df['Symbol'].unique())
print(current_constituents)
current_date = datetime.today()
previous_month_end = current_date.replace(day=1) - pd.Timedelta(days=1)
date_range = pd.date_range(start='2010-01-01', end=previous_month_end, freq='M')
date_range = date_range[::-1]

monthly_constituents = monthly_constituents.append({'Date': current_date, 'Constituents': current_constituents.copy()}, ignore_index=True)
for date in date_range:
    for index, row in changes_df.iterrows():
        change_month = pd.to_datetime(row['Date']).dt.month.item()
        change_year = pd.to_datetime(row['Date']).dt.year.item()
        if (change_year == date.year and change_month == date.month):
            current_constituents.discard(row['Added']['Ticker'])
            current_constituents.add(row['Removed']['Ticker'])
    print(current_constituents)
    monthly_constituents = monthly_constituents.append({'Date': date, 'Constituents': current_constituents.copy()}, ignore_index=True)

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('monatliche_bestandteile', monthly_constituents)


In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    a = store.get('monatliche_bestandteile')['Constituents']
    print(a.info())
    print(a)

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    a = store.get('monatliche_bestandteile')['Constituents']
    print(a.describe())
    print(a)


In [ ]:
with pd.HDFStore(DATA_STORE) as store:

    tickers = store.get('monatliche_bestandteile')['Constituents']
    all_tickers = list()
    for ticker_set in tickers:
        all_tickers.append(ticker_set)
    res = set().union(*all_tickers)
    print(res)
    print(len(res))

    store.put('all_tickers', pd.Series(list(res)))

In [ ]:
ALPHA_VANTAGE_API_KEY = 'SINNAMIDDPGINMN0'

# Historische Kursdaten von Alpha Vantage abrufen
def get_historical_data(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}&outputsize=full'
    try:
        response = requests.get(url).json()
    except:
        return pd.DataFrame()
    print(str(response))
    error = False
    if (str(response).startswith("{'Error Message'")):
        error = True
        print(symbol)
    if (error):
        return pd.DataFrame()
    json_data = response['Weekly Adjusted Time Series']

    data = []
    for date, values in json_data.items():
        data.append({
            'date': date,
            'open': float(values['1. open']),
            'high': float(values['2. high']),
            'low': float(values['3. low']),
            'close': float(values['4. close']),
            'adjusted_close': float(values['5. adjusted close']),
            'volume': int(values['6. volume'])
        })
    
    df = pd.DataFrame(data)
    return df


In [ ]:
# #ticker ausgeben, für die kein abruf möglich war
# #ticker anzahl ausgeben, für die kein abruf möglich war

# failed_requests = 0
# failed_tickers = []

# with pd.HDFStore(DATA_STORE) as store:
#     tickers = store.get('all_tickers')
#     df = pd.DataFrame(columns=['prices'])
#     i = 1
#     for ticker in tickers:
#         if (i % 75 == 0):
#             sleep(70)
#         try:
#             data = get_historical_data(ticker)
#             df.loc[ticker] = {'prices': data}
#         except Exception as e:
#             print(f"Failed to retrieve data for ticker {ticker}: {e}")
#             failed_requests += 1
#             failed_tickers.append(ticker)
#         i += 1
#     store.put('historical_prices', df)

# print(f"Failed requests: {failed_requests}")
# print(f"Failed tickers: {failed_tickers}")

# #count funktion geht hier nicht


In [ ]:
# # Initialisiere die Fehlerzähler
# failed_requests = 0
# failed_tickers = []

# # Funktion, um zählen zu ermöglichen
# class ApiError(Exception):
#     pass

# with pd.HDFStore(DATA_STORE) as store:
#     tickers = store.get('all_tickers')
#     df = pd.DataFrame(columns=['prices'])
#     i = 1
#     for ticker in tickers:
#         if (i % 75 == 0):
#             sleep(70)
#         try:
#             data = get_historical_data(ticker)
#             if "Error Message" in data:
#                 raise ApiError("API error")
#             df.loc[ticker] = {'prices': data}
#         except ApiError as e:
#             print(f"Failed to retrieve data for ticker {ticker}: {e}")
#             failed_requests += 1
#             failed_tickers.append(ticker)
#         except Exception as e:
#             print(f"Unexpected error for ticker {ticker}: {e}")
#         else:
#             i += 1
#     store.put('historical_prices', df)

# print(f"Failed requests: {failed_requests}")
# print(f"Failed tickers: {failed_tickers}")


In [ ]:
# Initialisiere die Fehlerzähler
failed_requests = 0
failed_tickers = []

# Funktion, um zählen zu ermöglichen
class ApiError(Exception):
    pass

with pd.HDFStore(DATA_STORE) as store:
    tickers = store.get('all_tickers')
    df = pd.DataFrame(columns=['prices'])
    i = 1
    for ticker in tickers:
        if (i % 75 == 0):
            sleep(70)
        try:
            data = get_historical_data(ticker)
            if "Error Message" in data:
                raise ApiError("API error")
            df.loc[ticker] = {'prices': data}
        except ApiError as e:
            print(f"Failed to retrieve data for ticker {ticker}: {e}")
            failed_requests += 1
            failed_tickers.append(ticker)
        except Exception as e:
            print(f" error for {ticker}: {e}")
        i += 1
    store.put('historical_prices', df)
#Fehlerzählen geht nicht
print(f"Failed requests: {failed_requests}")
print(f"Failed tickers: {failed_tickers}")


In [ ]:
# #mit fehlerzähler
# # Initialisiere die Fehlerzähler
# failed_requests = 0
# failed_tickers = []

# # Funktion, um Fehler zählen zu ermöglichen; Zählen funktioniert leider nicht; Aber Symbole
# class ApiError(Exception):
#     pass

# with pd.HDFStore(DATA_STORE) as store:
#     tickers = store.get('all_tickers')
#     df = pd.DataFrame(columns=['prices'])
#     i = 1
#     for ticker in tickers:
#         if (i % 75 == 0):
#             sleep(70)
#         try:
#             data = get_historical_data(ticker)
#             if "Error Message" in data:
#                 raise ApiError("API error")
#             df.loc[ticker] = {'prices': data}
#         except ApiError as e:
#             print(f"Failed to retrieve data for ticker {ticker}: {e}")
#             failed_requests += 1
#             failed_tickers.append(ticker)
#         except Exception as e:
#             print(f"Unexpected error for ticker {ticker}: {e}")
#         i += 1
#     store.put('historical_prices', df)

# print(f"Failed requests: {failed_requests}")
# print(f"Failed tickers: {failed_tickers}")
# print(f"Failed tickers: {len(failed_tickers)}")


In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    prices = store.get('historical_prices')
    print(prices.loc['XLNX']['prices'])

In [ ]:
#Test Berechnung Beta mit yfinance
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf

# Funktion zum Abrufen von monatlichen Preisdaten
def get_monthly_data(ticker):
    data = yf.download(ticker, start="2000-01-01", end="2023-04-02", interval="1mo")
    data = data.dropna()
    data = data["Adj Close"]
    return data

# Daten für Apple (AAPL) und S&P 500 Index (^GSPC) abrufen -> bei yfinance ist Abruf GSPC möglich
ticker_data = get_monthly_data("AAPL")
sp500_data = get_monthly_data("^GSPC")

# Monatliche Renditen berechnen
stock_returns = ticker_data.pct_change().dropna()
sp500_returns = sp500_data.pct_change().dropna()

# Rollierende Regression
window_size = 60  # 5 Jahre (12 Monate * 5 Jahre)
rolling_betas = []
rolling_dates = []

for i in range(len(stock_returns) - window_size + 1):
    y = stock_returns.iloc[i:i+window_size]
    X = sp500_returns.iloc[i:i+window_size]
    X = sm.add_constant(X)  # Konstante, um den Alpha-Koeffizienten zu schätzen
    model = sm.OLS(y, X).fit()
    beta = model.params["Adj Close"]
    rolling_betas.append(beta)
    rolling_dates.append(y.index[-1])

rolling_betas = pd.Series(rolling_betas, index=rolling_dates)

print(rolling_betas)

HIER NOCH VERGLEICH DER ABRUFE DURCH Y FINANCE FAILED
zeitraum ist extra länger eingestellt als bei wikipedia, weil für beta berechnung z.b. historische daten von vorher benötigt werden

In [ ]:
#yfinance liefert trotz beschränkung des Zeitraums für 99 Ticker keine Daten

# import yfinance as yf

# def get_historical_data_yf(ticker, start_date='2010-01-01', end_date=None):
#     if end_date is None:
#         end_date = datetime.today().strftime('%Y-%m-%d')
#     try:
#         stock_data = yf.download(ticker, start=start_date, end=end_date, interval='1wk', auto_adjust=True)
#         return stock_data
#     except Exception as e:
#         print(f"Error fetching data for {ticker}: {e}")
#         return pd.DataFrame()
# failed_requests = 0
# failed_tickers = []

# with pd.HDFStore(DATA_STORE) as store:
#     tickers = store.get('all_tickers')
#     df = pd.DataFrame()

#     for ticker in tickers:
#         data = get_historical_data_yf(ticker)
#         if data.empty:
#             failed_requests += 1
#             failed_tickers.append(ticker)
#         else:
#             data['ticker'] = ticker
#             df = df.append(data)

#     store.put('historical_prices_yf', df)

# print(f"Failed requests: {failed_requests}")
# print(f"Failed tickers: {failed_tickers}")


In [ ]:
# import yfinance as yf

# class YFinanceError(Exception):
#     pass

# def get_yfinance_data(ticker):
#     try:
#         data = yf.download(ticker, start="2000-01-01", end="2023-04-02", interval="1wk")
#         data = data.dropna()
#         data = data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
#         if data.empty:
#             raise YFinanceError("No data found")
#         return data
#     except Exception as e:
#         print(f"Failed to retrieve data for ticker {ticker}: {e}")
#         raise YFinanceError("Failed to download")

# failed_requests = 0
# failed_tickers = []

# with pd.HDFStore(DATA_STORE) as store:
#     tickers = store.get('all_tickers')
#     df = pd.DataFrame(columns=['prices'])
#     i = 1
#     for ticker in tickers:
#         try:
#             data = get_yfinance_data(ticker)
#             df.loc[ticker] = {'prices': data}
#         except YFinanceError as e:
#             failed_requests += 1
#             failed_tickers.append(ticker)
#         i += 1
#     store.put('historical_prices_yf', df)

# print(f"Failed requests: {failed_requests}")
# print(f"Failed tickers: {failed_tickers}")


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import requests
import json

api_key = 'SINNAMIDDPGINMN0'

# Hinzufügen von Fehlerzählern
fetch_errors = 0
calc_errors = 0

#   Abrufen von monatlichen Zeitreihendaten
def get_monthly_data(symbol, api_key):
    global fetch_errors  # Zugriff auf globale Variable fetch_errors

    base_url = "https://www.alphavantage.co/query"
    function = "TIME_SERIES_MONTHLY_ADJUSTED"
    outputsize = "full"
    datatype = "json"

    params = {
        "function": function,
        "symbol": symbol,
        "apikey": api_key,
        "outputsize": outputsize,
        "datatype": datatype
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if "Error Message" in data:
        print("Fehler bei Datenabruf für " + symbol)
        fetch_errors += 1  # Fehlerzähler für den Abruf erhöhen

        return pd.DataFrame()

    df = pd.DataFrame(data["Monthly Adjusted Time Series"]).T
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df["5. adjusted close"].astype(float)

    return df

stock_list = []
with pd.HDFStore(DATA_STORE) as store:
    tickers = store.get('all_tickers')
    for ticker in tickers:
        stock_list.append(ticker)

sp500_data = get_monthly_data("SPY", api_key) #Spy ETF statt Index
betas = pd.DataFrame(columns=['ticker', 'rolling_betas'])

for ticker in stock_list:

    try:
        # Daten für Aktien und S&P 500 ETF (SPY) abrufen
        ticker_data = get_monthly_data(ticker, api_key)

        # Startdatum auf den späteren Start von Aktien oder SPY setzen
        start_date = max(ticker_data.index.min(), sp500_data.index.min())

        # Daten auf gemeinsamen Zeitraum beschränken
        ticker_data = ticker_data[start_date:]
        sp500_data_last = sp500_data[start_date:]

        # Monatliche Renditen berechnen
        meta_returns = ticker_data.pct_change().dropna()
        sp500_returns = sp500_data_last.pct_change().dropna()

        # Rollierende Regression für 5 Jahres Beta
        window_size = 60  # 5 Jahre (12 Monate * 5 Jahre)
        rolling_betas = []
        rolling_dates = []

        for i in range(len(meta_returns) - window_size + 1):
            y = meta_returns.iloc[i:i+window_size]
            X = sp500_returns.iloc[i:i+window_size]
            X = sm.add_constant(X)  # Schätzen des Alpha-Koeffizienten
            model = sm.OLS(y, X).fit()
            beta = model.params["5. adjusted close"]
            rolling_betas.append(beta)
            rolling_dates.append(y.index[-1])
    except:
        print("Fehler bei Berechnung des Betas für " + ticker)
        calc_errors += 1  # Fehlerzähler für die Berechnung erhöhen

    rolling_betas = pd.Series(rolling_betas, index=rolling_dates)
    betas = betas.append({'ticker': ticker, 'rolling_betas': rolling_betas}, ignore_index=True)

    print(f"Fehler beim Abrufen der Daten: {fetch_errors}")
    print(f"Fehler bei der Berechnung der Beta-Werte: {calc_errors}")
#Zählen der Fehler beim Abruf + bei Berechnung der Beta Werte

In [ ]:
# #alternativ
# for ticker in stock_list:
#     try:
#         # Daten für Aktien und S&P 500 ETF (SPY) abrufen
#         ticker_data = get_monthly_data(ticker, api_key)
#     except:
#         print("Fehler beim Abrufen der Daten für " + ticker)
#         fetch_errors += 1
#         continue

#     try:
#         # Startdatum auf den späteren Start von Aktien oder SPY setzen
#         start_date = max(ticker_data.index.min(), sp500_data.index.min())

#         # Daten auf gemeinsamen Zeitraum beschränken
#         ticker_data = ticker_data[start_date:]
#         sp500_data_last = sp500_data[start_date:]

#         # Monatliche Renditen berechnen
#         meta_returns = ticker_data.pct_change().dropna()
#         sp500_returns = sp500_data_last.pct_change().dropna()

#         # Rollierende Regression für 5 Jahres Beta
#         window_size = 60  # 5 Jahre (12 Monate * 5 Jahre)
#         rolling_betas = []
#         rolling_dates = []

#         for i in range(len(meta_returns) - window_size + 1):
#             y = meta_returns.iloc[i:i+window_size]
#             X = sp500_returns.iloc[i:i+window_size]
#             X = sm.add_constant(X)  # Schätzen des Alpha-Koeffizienten
#             model = sm.OLS(y, X).fit()
#             beta = model.params["5. adjusted close"]
#             rolling_betas.append(beta)
#             rolling_dates.append(y.index[-1])

#         rolling_betas = pd.Series(rolling_betas, index=rolling_dates)
#         betas = betas.append({'ticker': ticker, 'rolling_betas': rolling_betas}, ignore_index=True)
#     except:
#         print("Fehler bei Berechnung des Betas für " + ticker)
#         calc_errors += 1

# print(f"Fehler beim Abrufen der Daten: {fetch_errors}")
# print(f"Fehler bei der Berechnung der Beta-Werte: {calc_errors}")


In [ ]:
print(betas.head())


In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('betas', betas)


In [ ]:
#Zusammenfügen der errechneten Beta Werte mit dem Datenset
with pd.HDFStore(DATA_STORE) as store:
    prices = store.get('historical_prices')
    betas = store.get('betas').set_index('ticker')
    for index, row in betas.iterrows():
        merge_prices = prices.loc[index]['prices'].set_index('date')
        merge_prices.index = pd.to_datetime(merge_prices.index)
        row.index = pd.to_datetime(row.index)
        merged = pd.merge_asof(merge_prices, row, left_index=True, right_index=True)

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    prices = store.get('historical_prices')
    betas = store.get('betas').set_index('ticker')
    prices_merged = prices.copy()
    for ticker in prices.index.values:
        print(ticker)
        merge_betas = betas.loc[ticker]['rolling_betas'].rename('betas')
        if (not merge_betas.empty):
            try:
                merge_prices = prices.loc[ticker]['prices'].set_index('date')
                merge_prices.index = pd.to_datetime(merge_prices.index)
                merge_prices = merge_prices.sort_index()
                merged = pd.merge_asof(merge_prices, merge_betas, left_on='date', right_index=True)
                prices_merged.loc[ticker]['prices'] = merged
            except:
                print("failed: " + ticker)

In [ ]:
meta = prices_merged.loc['META']['prices']

In [ ]:
prices.info()

In [ ]:
print(merged.iloc[1000])

In [ ]:
betas.loc['AAPL'].index

In [ ]:
merged.tail()

In [ ]:
#Plot Beta für bestimmte Aktie
betas.iloc[561]['rolling_betas'].plot()

In [ ]:
import datetime
import matplotlib.pyplot as plt

# Erstellen eines Plots für den Beta-Wert von Apple
betas.iloc[561]['rolling_betas'].plot()

# Festlegen des Zeitbereichs 
plt.xlim([datetime.date(2012, 1, 1), datetime.date(2023, 1, 8)])

# Festlegen des Wertebereichs für die y-Achse auf 0,8 bis 1,4
plt.ylim([0.7, 1.4])

# Anzeigen des Plots
plt.show()

In [ ]:
count = 0
for index, row in betas.iterrows():
    if (row['rolling_betas'].empty):
        count += 1
print(count)

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    print(store.keys())


In [ ]:
# `prices_merged` DataFrame im HDFStore speichern
with pd.HDFStore(DATA_STORE, mode='a') as store:
    store.put('prices_merged', prices_merged)

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    data = store.get('prices_merged')

In [ ]:
print(monthly_constituents.head())


In [ ]:
# Visualisierung der Anzahl an Bestandteilen im Zeitverlauf
monthly_constituents['Num_Constituents'] = monthly_constituents['Constituents'].apply(len)


import matplotlib.pyplot as plt

monthly_constituents['Num_Constituents'].plot(figsize=(10, 5))
plt.title("Anzahl der S&P 500 Aktien im Laufe der Zeit")
plt.ylabel("Anzahl der Aktien")
plt.xlabel("Datum")
plt.show()


In [ ]:
#Wann war welche Aktie im S&P500
def get_sp500_tickers_on_date(date, monthly_constituents):
    tickers_on_date = None
    for idx, row in monthly_constituents.iterrows():
        if row['Date'] <= date:
            tickers_on_date = row['Constituents']
            break
    return tickers_on_date


monthly_constituents_df = pd.DataFrame(columns=['Date', 'Constituents'])

start_date = '2009-01-01'
end_date = '2023-12-31'
date_range = pd.date_range(start=start_date, end=end_date)

for date in date_range:
    tickers_on_date = get_sp500_tickers_on_date(date, monthly_constituents)
    monthly_constituents_df = monthly_constituents_df.append({'Date': date, 'Constituents': tickers_on_date}, ignore_index=True)

# Speichern des DataFrame im HDFStore
DATA_STORE = 'sp.h5'
with pd.HDFStore(DATA_STORE) as store:
    store.put('monthly_constituents', monthly_constituents_df)
    monthly_constituents_backtest_df = monthly_constituents_df

    store.put('monthly_constituents_backtest', monthly_constituents_backtest_df)


with pd.HDFStore(DATA_STORE) as store:
    print(store.keys())



In [ ]:
monthly_constituents.set_index('Date', inplace=True)

# Visualisierung der Anzahl an Bestandteilen im Zeitverlauf
monthly_constituents['Num_Constituents'] = monthly_constituents['Constituents'].apply(len)

import matplotlib.pyplot as plt

monthly_constituents['Num_Constituents'].plot(figsize=(10, 5))
plt.title("Anzahl der S&P 500 Aktien im Laufe der Zeit")
plt.ylabel("Anzahl der Aktien")
plt.xlabel("Datum")
plt.show()
